Copyright 2022 Michael Govaerts, Anike Braun, and Abigail Harrison

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated 
documentation files (the "Software"), to deal in the Software without restriction, including without limitation 
the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, 
and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions
of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF
CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS

In [1]:
from adaptive_backend import *
from Election_Simulation import *

In [2]:
#Gathers Election data from Simulation_Input.txt to create a simulated election to audit
#In a real audit, upload the necessary files to the directory and do not run this code block
simulationData, margins = readInput()
numBallots, overvotes1, undervotes1, overvotes2, undervotes2, riskLimit, num, gamma = dataToValues(simulationData)
margin = margins[0][0]

if (os.path.exists("2020_CT_Election_Data.json")):
    #Imports JSON file with election population
    inputFile = open(os.path.join(sys.path[0], "2020_CT_Election_Data.json"), "r")
    jsonFile = json.load(inputFile)
    if jsonFile is None:
        raise SyntaxError("Something is wrong with the JSON file. Please check it and try again.")
else:
    jsonFile = None

E1 = Election(numBallots, margin, overvotes1, undervotes1, overvotes2, undervotes2, riskLimit, gamma, 1, jsonFile)
#Creates the ballot manifest, tabulation, and CVRs
electionSetup(E1)

Election setup:
CVR1 created
Manifest created
CVR2 created
Tabulation created


In [3]:
#This deletes the subdirectory of lazy_rla_cvr which is where all temporary files are stored
#You must use this code block
removeWorkingDir()

In [4]:
#This is the first seed from dice rolling ceremony, used to select batches
#There will be a second seed used to select ballots
seed1 = 2368607141
selectedBatches = batchSelect('electionManifest.csv','electionTabulation.csv', seed1)

numBallots, winnerBallots, runnerupBallots, margin
100000 52390 47610 4.780000000000001
137 ballots to audit
ballots selected from 126 different batches


In [5]:
#In an actual audit skip this codeblock
#This is used to populate fake CVR files using data from 2020 Presidential election in CT
lazyCVR_files = lazyCVR_gen(selectedBatches['batchesToAudit'])

In [6]:
#Second seed used to select ballots. Should be independent from first dice rolling ceremony
#Generates blank CVR files for your own interpretation. Fill them in before running the next code block.
seed2 = 9113645654
auditCVR_blank = ballotSelect(lazyCVR_files, selectedBatches['ballotsPerBatchAudit'], selectedBatches['ballotsPerBatchTotal'], seed2)

In [7]:
#auditCVR_check is list of files with correct 'manual interpretations' filled out
#This step not needed in a real audit, this is to run quickly
auditCVR_check = ballotSelect_check(lazyCVR_files, selectedBatches['ballotsPerBatchAudit'], selectedBatches['ballotsPerBatchTotal'], seed2)

Generating files since no manual interpretation.


In [8]:
#In a real audit, set the flag = 1
#flag = 0 creates files for interpretation, while flag = 1 means you are filling in your own interpretations
flag = 0

In [9]:
#Calculate risk give ballots and discrepancies
riskLimit = 0.05 #Desired risk limit
riskLevel = calculateRisk(auditCVR_check, lazyCVR_files, 'electionTabulation.csv', 'electionManifest.csv', riskLimit, seed1, seed2, flag)
print('risk limit: ' + str(riskLevel))

In that round, risk limit = 0.15432515473255354
Risk limit not met in current round, starting next round.

numBallots, winnerBallots, runnerupBallots, margin
100000 52390 47610 4.780000000000001
202 ballots to audit
ballots selected from 176 different batches
Generating files since no manual interpretation.
In that round, risk limit = 0.002599079052324125
risk limit: 0.002599079052324125
